<a href="https://colab.research.google.com/github/armaank/dbn/blob/main/dbn/Explore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## RepresentationSpace - Discovering Interpretable GAN Controls for Architectural Image Synthesis

Using [Ganspace](https://github.com/harskish/ganspace) to explore fixed latent directions in a StyleGAN2 model trained trained on the ArchML dataset

### Instructions and Setup

1) Click the play button of the blocks titled "Initialization" and wait for it to finish the initialization.

2) Click the play button to on the block titled "Load Model". This block will take a little bit (~1-2 minutes) to run.

3) In the section named "Explore RepresentationSpace", generate samples, and play with the sliders. In the next block generate videos.


In [1]:
%%capture
#@title Initialization - Setup
# Clone git
%reset -f c
%tensorflow_version 1.x
%rm -rf dbn
!git clone https://github.com/armaank/dbn
%cd dbn/generative/
%ls

In [2]:
#@title Initialization - Download Models
%%capture
%%sh
python3 get_models.py
python3 get_directions.py

success
success


tcmalloc: large alloc 1621008384 bytes == 0x631ee000 @  0x7fabddbaf1e7 0x4a3940 0x5b438c 0x5b46f7 0x59afff 0x515655 0x593dd7 0x511e2c 0x549576 0x593fce 0x511e2c 0x549576 0x593fce 0x511e2c 0x593dd7 0x5118f8 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e 0x7fabdd7acc87 0x5b621a
/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
#@title Initilization - Install Requirements
%%capture

from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 200})'''))
!pip install fbpca boto3
!git submodule update --init --recursive
!python -c "import nltk; nltk.download('wordnet')"
%cd ./ganspace/

from IPython.utils import io
import torch
import PIL
import numpy as np
import ipywidgets as widgets
from PIL import Image
import imageio
from models import get_instrumented_model
from decomposition import get_or_compute
from config import Config
from skimage import img_as_ubyte

# Speed up computation
torch.autograd.set_grad_enabled(False)
torch.backends.cudnn.benchmark = True
# Custom OPs no longer required
#!pip install Ninja
#%cd models/stylegan2/stylegan2-pytorch/op
#!python setup.py install
#!python -c "import torch; import upfirdn2d_op; import fused; print('OK')"
#%cd "/content/ganspace"

<IPython.core.display.Javascript object>

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/content/dbn/generative/ganspace
StyleGAN2: Optimized CUDA op FusedLeakyReLU not available, using native PyTorch fallback.
StyleGAN2: Optimized CUDA op UpFirDn2d not available, using native PyTorch fallback.


In [5]:
#@title Load Model

# model = "Adaily-B" #@param ["Adaily-A", "Adaily-B"]
# num_components =  80#@param {type:"number"}
# layer = 'style'#@param ["style","input","convs","upsamples","noises"]

model = 'Adaily-B'
num_components = 80
layer = 'style'
model_class = model # this is the name of model
# if model == 'Adaily-A':
#   model_class = 'Adaily_A'
# if model == 'Adaily-B':
#   model_class = 'Adaily_B'

model_name = 'StyleGAN2'

# !python visualize.py --model $model_name --class $model_class --use_w --layer=style -c $num_components
from IPython.display import display, clear_output
from ipywidgets import fixed
#@title Load Model and Component 

config = Config(
  model='StyleGAN2',
  layer=layer,
  output_class=model_class,
  components=num_components,
  use_w=True,
  batch_size=5_000, # style layer quite small
)

inst = get_instrumented_model(config.model, config.output_class,
                              config.layer, torch.device('cuda'), use_w=config.use_w)

path_to_components = get_or_compute(config, inst)
model = inst.model

# named_directions = {} #init named_directions dict to save directions
named_directions = {'Site - Drawing': [0, 0, 3], 'Image - Drawing': [0, 0, 18], 'Shaded - Hatched': [0, 6, 10], 'Light - Dark': [0, 14, 18], 'Outline - Poche': [0, 7, 14], 'Subdivided - Open': [1, 8, 10], 'Interior Color': [2, 11, 18], 'Small - Large': [2, 4, 8], 'Elevation - Plan': [3, 0, 18], 'Paper Color': [4, 12, 18], 'Shadows': [4, 12, 14], 'Tall - Long': [4, 0, 18], 'Section - Plan': [5, 0, 18], 'Shaded - Outline': [6, 10, 18], 'Closed - Open': [7, 6, 7], 'Multiple - Single': [7, 0, 4], 'Detail': [13, 6, 9]}
comps = np.load(path_to_components)
lst = comps.files
latent_dirs = []
latent_stdevs = []

load_activations = False

for item in lst:
    if load_activations:
      if item == 'act_comp':
        for i in range(comps[item].shape[0]):
          latent_dirs.append(comps[item][i])
      if item == 'act_stdev':
        for i in range(comps[item].shape[0]):
          latent_stdevs.append(comps[item][i])
    else:
      if item == 'lat_comp':
        for i in range(comps[item].shape[0]):
          latent_dirs.append(comps[item][i])
      if item == 'lat_stdev':
        for i in range(comps[item].shape[0]):
          latent_stdevs.append(comps[item][i])
            

#load one at random 
num = np.random.randint(20)
if num in named_directions.values():
  print(f'Direction already named: {list(named_directions.keys())[list(named_directions.values()).index(num)]}')

random_dir = latent_dirs[num]
random_dir_stdev = latent_stdevs[num]

# print(f'Loaded Component No. {num}')
print(f'Model Loaded')


../models/Adaily-B/torch_official/stylegan2_Adaily_1024.pt
Not cached
[11.07 22:29] Computing stylegan2-Adaily-B_style_ipca_c80_n300000_w.npz
Reusing InstrumentedModel instance
Using W latent space
Feature shape: torch.Size([1, 512])
B=5000, N=300000, dims=512, N/dims=585.9


Fitting batches (NB=5000): 100%|##########| 60/60 [00:33<00:00,  1.77it/s]


Total time: 0:00:41.472850
Model Loaded



# Explore RepresentationSpace

Using the UI, you can explore the latent directions by selecting their name. The variable Seed controls the starting image. The Truncation slider controls the quality of the image sample, .7 is a good starting point. Distance is the main slider, it controls the strength/emphasis of the component.


In [10]:
#@title Visualize Named Directions


# Taken from https://github.com/alexanderkuk/log-progress
def log_progress(sequence, every=1, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

def name_direction(sender):
  if not text.value:
    print('Please name the direction before saving')
    return
    
  if num in named_directions.values():
    target_key = list(named_directions.keys())[list(named_directions.values()).index(num)]
    print(f'Direction already named: {target_key}')
    print(f'Overwriting... ')
    del(named_directions[target_key])
  named_directions[text.value] = [num, start_layer.value, end_layer.value]
  save_direction(random_dir, text.value)
  for item in named_directions:
    print(item, named_directions[item])

def save_direction(direction, filename):
  filename += ".npy"
  np.save(filename, direction, allow_pickle=True, fix_imports=True)
  print(f'Latent direction saved as {filename}')

def display_sample_pytorch(seed, truncation, direction, distance, start, end, disp=True, save=None, noise_spec=None, scale=2,):
    # blockPrint()
    with io.capture_output() as captured:
      w = model.sample_latent(1, seed=seed).cpu().numpy()

      model.truncation = truncation
      w = [w]*model.get_max_latents() # one per layer
      for l in range(start, end):
          w[l] = w[l] + direction * distance * scale

      #save image and display
      out = model.sample_np(w)
      final_im = Image.fromarray((out * 255).astype(np.uint8)).resize((500,500),Image.LANCZOS)

    if disp:
      display(final_im)
    if save is not None:
      if disp == False:
        print(save)
      final_im.save(f'out/{seed}_{save:05}.png')

def generate_mov(seed, truncation, direction_vec,  layers, n_frames, out_name = 'out', scale = 2, noise_spec = None, loop=True):
  """Generates a mov moving back and forth along the chosen direction vector"""
  # Example of reading a generated set of images, and storing as MP4.
  %mkdir out
  movieName = f'out/{out_name}.mp4'
  offset = -10
  step = 20 / n_frames
  imgs = []
  for i in log_progress(range(n_frames), name = "Generating frames"):
    print(f'\r{i} / {n_frames}', end='')
    w = model.sample_latent(1, seed=seed).cpu().numpy()

    model.truncation = truncation
    w = [w]*model.get_max_latents() # one per layer
    for l in layers:
      if l <= model.get_max_latents():
          w[l] = w[l] + direction_vec * offset * scale

    #save image and display
    out = model.sample_np(w)
    final_im = Image.fromarray((out * 255).astype(np.uint8))
    imgs.append(out)
    #increase offset
    offset += step
  if loop:
    imgs += imgs[::-1]
  with imageio.get_writer(movieName, mode='I') as writer:
    for image in log_progress(list(imgs), name = "Creating animation"):
        writer.append_data(img_as_ubyte(image))

vardict = list(named_directions.keys())
select_variable = widgets.Dropdown(
    options=vardict,
    value=vardict[0],
    description='Select variable:',
    disabled=False,
    button_style=''
)

def set_direction(b):
    clear_output()
    random_dir = latent_dirs[named_directions[select_variable.value][0]]
    start_layer = named_directions[select_variable.value][1]
    end_layer = named_directions[select_variable.value][2]
    print(start_layer, end_layer)
    out = widgets.interactive_output(display_sample_pytorch, {'seed': seed, 'truncation': truncation, 'direction': fixed(random_dir), 'distance': distance, 'scale': scale, 'start': fixed(start_layer), 'end': fixed(end_layer)})
    display(select_variable)
    display(ui, out)


random_dir = latent_dirs[named_directions[select_variable.value][0]]
start_layer = named_directions[select_variable.value][1]
end_layer = named_directions[select_variable.value][2]
seed = np.random.randint(0,100000)
style = {'description_width': 'initial'}

seed = widgets.IntSlider(min=0, max=100000, step=1, value=seed, description='Seed: ', continuous_update=False)
truncation = widgets.FloatSlider(min=0, max=2, step=0.1, value=0.7, description='Truncation: ', continuous_update=False)
distance = widgets.FloatSlider(min=-10, max=10, step=0.1, value=0, description='Distance: ', continuous_update=False, style=style)
scale = widgets.FloatSlider(min=0, max=10, step=0.05, value=1, description='Scale: ', continuous_update=False)

bot_box = widgets.HBox([seed, truncation, distance])
ui = widgets.VBox([bot_box])
out = widgets.interactive_output(display_sample_pytorch, {'seed': seed, 'truncation': truncation, 'direction': fixed(random_dir), 'distance': distance, 'scale': scale, 'start': fixed(start_layer), 'end': fixed(end_layer)})

display(select_variable)
display(ui, out)

select_variable.observe(set_direction, names='value')

6 9


Dropdown(description='Select variable:', index=16, options=('Site - Drawing', 'Image - Drawing', 'Shaded - Hat…

Output()

In [13]:
#@title Generate Video from Representation (Optional)
direction_name = "Image - Drawing" #@param ["Site - Drawing", "Image - Drawing", "Shaded - Hatched", "Light - Dark", "Outline - Poche", "Subdivided - Open", "Interior Color", "Small - Large", "Elevation - Plan", "Paper Color", "Shadows", "Tall - Long", "Section - Plan", "Closed - Open" , "Multiple - Single", "Detail" ] {allow-input: true}
num_frames =  5 #@param {type:"number"}
truncation = 0.8 #@param {type:"number"}

num_samples = num_frames
assert direction_name in named_directions, \
  f'"{direction_name}" not found, please save it first using the cell above.'

loc = named_directions[direction_name][0]
for i in range(num_samples):
  s = np.random.randint(0, 10000)
  generate_mov(seed = s, truncation = 0.8, direction_vec = latent_dirs[loc], scale = 2, layers=range(named_directions[direction_name][1], named_directions[direction_name][2]), n_frames = 20, out_name = f'{model_class}_{direction_name}_{i}', loop=True)

print('Video saved to ./ganspace/out/')

mkdir: cannot create directory ‘out’: File exists


19 / 20

mkdir: cannot create directory ‘out’: File exists


19 / 20

mkdir: cannot create directory ‘out’: File exists


...

mkdir: cannot create directory ‘out’: File exists


19 / 20

mkdir: cannot create directory ‘out’: File exists


19 / 20

Video saved to ./ganspace/out/
